In [1]:
import pandas as pd
import importlib.util
import sklearn
import numpy as np
from pathlib import Path
import joblib
import os   

In [2]:
output_dir = "classification_results_2023/"
all_patient_data = pd.read_csv("Combined_AM_Data.csv")

In [3]:
all_patient_data.columns

Index(['id', 'date', 'age', 'sex', 'BMI', 'income', 'road_dist', 'cooking',
       'am_pef_org', 'y_am_pef', 'tempin', 'humidin', 'pm25in', 'co2in',
       'tempdiffin', 'humidiffin', 'pm25diffin', 'pm10', 'pm25', 'o3', 'no2',
       'co', 'so2', 'temp', 'windsd', 'humid', 'varp', 'dewpt', 'airp', 'seap',
       'solrhr', 'solramnt', 'grdt', 'class'],
      dtype='object')

In [4]:
# Loading classification.py
spec = importlib.util.spec_from_file_location('classification', r"Classificationv3.py")
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [5]:
predictColumn = "class"
minorityVariable = 0 
classes = all_patient_data[predictColumn].unique().tolist()

n_classes = len(classes)

if n_classes > 2 :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='saga', multi_class='multinomial')
else :
    logit_reg_model = sklearn.linear_model.LogisticRegression(solver='liblinear', multi_class='auto')

cls = [ "DecisionTree" , sklearn.tree.DecisionTreeClassifier(max_depth=4),
        "K-NN"              , sklearn.neighbors.KNeighborsClassifier(),
        "Logit Regression"  , logit_reg_model,
        "Naive Bayes"       , sklearn.gaussian_process.GaussianProcessClassifier()
];

In [6]:
EVALUATION_METRICS = ['Weighted Accuracy', 'Sensitivity/Recall', 'Precision_avg', 'F1_avg', 'Specificity', 'Precision', 'Precision_class1', 'F1', 'F1_class1']
REQUIRED_COLUMNS = ['income','road_dist','cooking','y_am_pef','tempin','humidin','pm25in','co2in','tempdiffin','humidiffin','pm25diffin','pm10','pm25','o3','no2','co','so2','temp','windsd','humid','varp','dewpt','airp','seap','solrhr','solramnt','grdt','class']

In [7]:
%run "Classificationv3.py"
%run "Balance.py"

In [8]:
def classification_func(file_name, predictCol, minorVar, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df):

    print('~~~~~~~~~~~~~~~~~', file_name,'~~~~~~~~~~~~~~~~~~~~~~')

    dataset_dir = Path(output_dir + patient_id+category_group +  "/diagrams/" + file_name + '/') 

    # Create Directory for this patient
    if not (dataset_dir.exists() and dataset_dir.is_dir()):
        os.makedirs(dataset_dir)

    save_path = output_dir+ patient_id+category_group +'/diagrams/' + file_name + '/sklearn_models_balanced_for_train_'

    CLASS_VAR = predictCol
    MINORITY_VAR = minorVar
    bestDT = module.Classify(patient_id_df, True, class_var=CLASS_VAR, minority_var=MINORITY_VAR, drawConfusionMatrix=True, classifiers=cls, scale=True, save_path=save_path, after_split=True)

    confusion_matrixes_file = save_path + "confusion_mat_" + category_group + "_"+ file_name
    module.save_confusion_matrix(bestDT[1], confusion_matrixes_file)
    roc_curve_file = save_path + "roc_curve_" + category_group + "_"+ file_name
    module.save_Draw_Roc(bestDT[2], roc_curve_file)
    if len(bestDT[3])!=0:
      featureImportance_path = save_path + "important_mat_" + category_group + "_"+ file_name
      module.save_Draw_FeatureImportanceMatrix(patient_id_df,bestDT[3],cls[0], featureImportance_path, True)

    model_path = save_path + "bestModel_" + category_group + "_"+ file_name + ".joblib"
    joblib.dump(bestDT[4], model_path)

    df_evaluation_results[file_name] = bestDT[0]
    df_evaluation_results.to_csv(output_dir+patient_id+category_group+"/sklearn_models_balanced_for_train_metrics_"+category_group + ".csv")


In [9]:
def runner_decision_tree():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Decision Tree
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = '1DecisionTree'
        cls = ["DecisionTree", sklearn.tree.DecisionTreeClassifier(max_depth=4)]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_knn():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # KNN
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = '2K-NN'
        cls = ["K-NN", sklearn.neighbors.KNeighborsClassifier()]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_lr():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Logistic Regression
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'LogisticRegression'
        print(logit_reg_model)
        cls = ["Logit Regression", logit_reg_model]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_svm():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # SVM
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'SVM'
        cls = ["SVM", sklearn.svm.SVC(kernel="linear")]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)

def runner_nb():
    ids = all_patient_data['id'].unique().tolist()
    for id in ids:
        patient_id = id + "/"
        file_name = id
        patient_id_df = all_patient_data[all_patient_data['id'] == id]
        filtered_df = patient_id_df.filter(REQUIRED_COLUMNS)
        patient_id_df = filtered_df.copy()
        
        # Naive Bayes
        df_evaluation_results = pd.DataFrame(index=EVALUATION_METRICS)
        category_group = 'NaiveBayes'
        cls = ["Naive Bayes", sklearn.gaussian_process.GaussianProcessClassifier()]
        classification_func(file_name, predictColumn, minorityVariable, category_group, cls, output_dir, df_evaluation_results, patient_id, patient_id_df)


In [10]:
runner_decision_tree()

~~~~~~~~~~~~~~~~~ SB-001 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  125  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.01030796580016613  ~~~~~~~
~~~~~~~ Validation Loss: 0.011773115023970604  ~~~~~~~
2/2 [==============================] - 1s 13ms/step
~~~~~~~ value_counts: ~~~~~~~
1    86
0    78
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  164  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  125  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.009755276143550873  ~~~~~~~
~~~~~~~ Validation Loss: 0.013918748125433922  ~~~~~~~
2/2 [==============================] - 0s 6ms/step
~~~~~~~ value_counts: ~~~~~~~
1    86
0    78
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  164  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  126  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.009176133200526237  ~~~~~~~
~~~~~~~ Validation Loss: 0.012631961144506931  ~~~~~~~
2/2 [==============================] - 0s 5ms/step
~~~

Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
---------------DrawFeatureImportanceMatrix------------
------------DrawFeatureImportanceMatrix end----------------
~~~~~~~~~~~~~~~~~ SB-003 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.020188476890325546  ~~~~~~~
~~~~~~~ Validation Loss: 0.028955882415175438  ~~~~~~~
1/1 [==============================] - 0s 454ms/step
~~~~~~~ value_counts: ~~~~~~~
1    91
0    58
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  149  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.010569949634373188  ~~~~~~~
~~~~~~~ Validation Loss: 0.0214027501642704  ~~~~~~~
1/1 [==============================] - 0s 373ms/step
~~~~~~~ value_counts: ~~~~~~~
1    88
0    64
Name: class, dtype: int64
~~~~~~~~~~ Number of instances

Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
---------------DrawFeatureImportanceMatrix------------
------------DrawFeatureImportanceMatrix end----------------
~~~~~~~~~~~~~~~~~ SB-008 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  118  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.007760611362755299  ~~~~~~~
~~~~~~~ Validation Loss: 0.01613534614443779  ~~~~~~~
1/1 [==============================] - 0s 211ms/step
~~~~~~~ value_counts: ~~~~~~~
1    88
0    60
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  148  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  119  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.015625840052962303  ~~~~~~~
~~~~~~~ Validation Loss: 0.035300929099321365  ~~~~~~~
1/1 [==============================] - 0s 143ms/step
~~~~~~~ value_counts: ~~~~~~~
1    98
0    42
Name: class, dtype: int64
~~~~~~~~~~ Number of instance

...

++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
---------------DrawFeatureImportanceMatrix------------
------------DrawFeatureImportanceMatrix end----------------
~~~~~~~~~~~~~~~~~ SB-089 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  32  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.011022713035345078  ~~~~~~~
~~~~~~~ Validation Loss: 0.055588770657777786  ~~~~~~~
1/1 [==============================] - 0s 207ms/step
~~~~~~~ value_counts: ~~~~~~~
1    24
0    16
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  40  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  33  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.01752055436372757  ~~~~~~~
~~~~~~~ Validation Loss: 0.02460821531713009  ~~~~~~~
1/1 [==============================] - 0s 344ms/step
~~~~~~~ value_counts: ~~~~~~~
0    26
1    20
Name: class, dtype: int64
~~~~~~~~~~ Number of instances af

Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
---------------DrawFeatureImportanceMatrix------------
------------DrawFeatureImportanceMatrix end----------------
~~~~~~~~~~~~~~~~~ SB-112 ~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.010879767127335072  ~~~~~~~
~~~~~~~ Validation Loss: 0.03488393500447273  ~~~~~~~
1/1 [==============================] - 0s 279ms/step
~~~~~~~ value_counts: ~~~~~~~
1    94
0    52
Name: class, dtype: int64
~~~~~~~~~~ Number of instances after balancing:  146  ~~~~~~~~~~
~~~~~~~~~~ Number of instances before balancing:  120  ~~~~~~~~~~
~~~~~~~ Training Loss: 0.010024706833064556  ~~~~~~~
~~~~~~~ Validation Loss: 0.02139320783317089  ~~~~~~~
1/1 [==============================] - 0s 271ms/step
~~~~~~~ value_counts: ~~~~~~~
1    93
0    54
Name: class, dtype: int64
~~~~~~~~~~ Number of instances

Classificationv3.py:353: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_xticklabels([''] + ['0', '1'], fontsize='xx-large')
Classificationv3.py:354: UserWarning: FixedFormatter should only be used together with FixedLocator
  axs[i].set_yticklabels([''] + ['0', '1'], fontsize='xx-large')


++++++++++++save_confusion_matrix end++++++++++++
(((((((((((((((((((((( draw droc ))))))))))))))))))))))))))))))
(((((((( draw droc end ))))))))
---------------DrawFeatureImportanceMatrix------------
------------DrawFeatureImportanceMatrix end----------------


In [11]:
# runner_knn()

In [12]:
# runner_lr()

In [13]:
# runner_svm()

In [14]:
# runner_nb()